In [ ]:
def createADS(workingDir):
    os.chdir(workingDir)
    #===========================================================================
    #Import relavent csv's
    #===========================================================================
    receiptsRAW_df=pd.read_csv('201904 sales receipts.csv')
    
    #===========================================================================
    #We want to create a column that tracks whether the order was made by a member or not
        #Replace missing customer ID's with 0
    #===========================================================================
    receiptsRAW_df=receiptsRAW_df.fillna(0)
    
    #===========================================================================
    #Create a binary column that tracks if the order was made by member
        #This will be needed when making the decision tree
    #===========================================================================
    receiptsRAW_df['LoyaltyMember']=receiptsRAW_df['customer_id'].apply(lambda x: 1 if x>0 else 0)
    
    
    NYC_WeatherRAW_df=pd.read_csv('NYC_weather.csv')
    ProductsRAW_df=pd.read_csv('product.csv')

    #===========================================================================
    #Columns to keep for each dataframe
    #===========================================================================
    receipts_colNames=['transaction_id','transaction_date','transaction_time','sales_outlet_id','instore_yn','product_id','quantity','line_item_amount','LoyaltyMember']
    Weather_colNames=['date','Avg_temp','Precip_inches_yesterday','wind_mph_avg']
    product_colNames=['product_id','product_group']
    
    #===========================================================================
    #function to drop unnecessary columns
    #===========================================================================
    def keepCols(input_df,cols_to_keep):
        Output_df = input_df[input_df.columns.intersection(cols_to_keep)]
        return Output_df

    #===========================================================================
    #Function calls
    #===========================================================================
    receipts_df=keepCols(receiptsRAW_df,receipts_colNames)
    Weather_df=keepCols(NYC_WeatherRAW_df,Weather_colNames)
    products_df=keepCols(ProductsRAW_df,product_colNames)





    #===========================================================================
    #Formatting the weather dataframe
    #===========================================================================
    #Shift precipitation one cell up; This will turn previous day's precipitation value into current day's precipitation
    Weather_df['Precip_inches_yesterday'] = Weather_df['Precip_inches_yesterday'].shift(-1).ffill()
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

    #Rename the column
    Weather_df = Weather_df.rename(columns={'Precip_inches_yesterday': 'Precip_inches'})

    Weather_df['date']=pd.to_datetime(Weather_df['date'])

    #===========================================================================
    #Formatting the receipts dataframe
    #===========================================================================
    #Convert store into dummy columns - one binary column per store
    receipts_df=pd.get_dummies(receipts_df,columns=['sales_outlet_id'], prefix='Store ')

    #Convert Y/N values to 1/0
    receipts_df['instore_yn'] = receipts_df['instore_yn'].apply(lambda x: 1 if x=='Y' else 0)

    receipts_df['transaction_date']=pd.to_datetime(receipts_df['transaction_date'])

    #Whole timestamp is not necessary and will get in the way of DecisionTree... 
    #The below is extracting the hour and dropping the original column
    receipts_df['Hour']=pd.to_numeric(receipts_df['transaction_time'].str[:2])
    receipts_df.drop('transaction_time', axis=1, inplace=True)  


    #===========================================================================
    #Merging products dataframe with the receipts dataframe
    #===========================================================================

    Receipts_ProdGrp_df=receipts_df.merge(products_df,left_on='product_id',right_on='product_id',how='left')
    

    #===========================================================================
    #Merging Receipts_ProdGrp_df dataframe with the weather dataframe
    #===========================================================================

    Receipts_ProdWthr_df=Receipts_ProdGrp_df.merge(Weather_df,left_on='transaction_date',right_on='date',how='left')
    Receipts_ProdWthr_df.drop('transaction_date', axis=1, inplace=True)  

    #===========================================================================
    #Seperate and create a pivot out of Transaction ID, product_group, and line_item_amount
    #===========================================================================

    pvt_cols=['transaction_id','line_item_amount','product_group']
    OrderAmt_byGrp_df=keepCols(Receipts_ProdWthr_df,pvt_cols)

    OrderAmt_byGrp_PivotDF = pd.pivot_table(OrderAmt_byGrp_df, values='line_item_amount', index=['transaction_id'],columns=['product_group'], aggfunc=np.sum,fill_value=0)

    #===========================================================================
    #Aggregate other cols on transaction id
    #===========================================================================

    neededCols_forAgg=['transaction_id','instore_yn','quantity','line_item_amount','Store _3','Store _5','Store _8','Hour','Avg_temp','Precip_inches','wind_mph_avg','LoyaltyMember']
    Agg_ADS=keepCols(Receipts_ProdWthr_df,neededCols_forAgg)

    OrdrLvl_ADS=Agg_ADS.groupby('transaction_id').agg({'instore_yn':'max', 'quantity':'sum', 'line_item_amount':'sum', 'Store _3':'max','Store _5':'max','Store _8':'max','Hour':'max','Avg_temp':'max','Precip_inches':'max','wind_mph_avg':'max','LoyaltyMember':'max'})
    OrdrLvl_ADS = OrdrLvl_ADS.rename(columns={'quantity': 'TotalQuantity','line_item_amount': 'TotalAmount'})

    #===========================================================================
    #Merge pivot with above aggregated table
    #===========================================================================

    Dtree_ADS=OrdrLvl_ADS.merge(OrderAmt_byGrp_PivotDF,left_on='transaction_id',right_on='transaction_id',how='left')
    return Dtree_ADS

In [ ]:
def Plot_DT(workingDir,DT_ADS):
    os.chdir(workingDir)
    
    feature_cols = ['instore_yn', 'TotalQuantity', 'TotalAmount', 'Store _3','Store _5','Store _8','Add-ons','Beverages','Food','Merchandise','Whole Bean/Teas']
    attr_var = DT_ADS[feature_cols] #Attributes
    target_Var=DT_ADS['LoyaltyMember'] #Target variable

    #===========================================================================
    #Split dataset into train/test (80/20)
    #===========================================================================
    attr_train, attr_test, target_train, target_test = train_test_split(attr_var, target_Var, test_size=0.2, random_state=0)

    #===========================================================================
    #Decision tree model
        #Chose max_depth of 3 to keep the tree small
    #===========================================================================
    dt_model = DecisionTreeClassifier(max_depth=3)

    #===========================================================================
    # Fit training dataset to model
    #===========================================================================
    dt_model = dt_model.fit(attr_train,target_train)

    #===========================================================================
    #Predict
    #===========================================================================
    target_preds = dt_model.predict(attr_test)

    #===========================================================================
    #Print metrics and plot
    #===========================================================================
    print("Accuracy:",metrics.accuracy_score(target_test, target_preds))

    targetNames=DT_ADS['LoyaltyMember'].unique().tolist()


    print("Max Depth:",dt_model.tree_.max_depth)

    fig = plt.figure(figsize=(25,20))
    _ = plot_tree(dt_model, 
              feature_names = feature_cols, 
              class_names = ['0','1'], 
              filled = True)
    plt.savefig('DecisionTree.png')

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import datetime as dt
from sklearn.tree import plot_tree
import os

pd.options.mode.chained_assignment = None  # default='warn'

def runEverything(workingDir):
    DT_ADS=createADS(workingDir)
    Plot_DT(workingDir,DT_ADS)

print("Please input the directory where the RAW files are stored: ")
directory_str=input()
print("Directory: "+ directory_str)
directory_str=directory_str+"\\"
runEverything(directory_str)
